In [ ]:
# default_exp scrape13F

# scrape13F

> Scrape investor holdings from 13F-HR SEC filings.

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
from nbdev import show_doc

In [ ]:
#export

import collections
import itertools
import numpy as np
import os
import re

from secscan import utils, dailyList, basicInfo, infoScraper

default13FDir = os.path.join(utils.stockDataRoot,'scraped13F')

13F-HR scraper class - scrape table of investor holdings from XML format in the SEC filing:

In [ ]:
#export

def findChildEndingWith(el,tagEnd) :
    "Finds first child of an XML element with tag ending in tagEnd (case insensitive)."
    tagEnd = tagEnd.lower()
    for child in el :
        if child.tag.lower().endswith(tagEnd) :
            return child
    return None

def findChildSeries(el,tagEnds) :
    "Finds a nested series of children by tag using findChildEndingWith"
    for tagEnd in tagEnds :
        el = findChildEndingWith(el,tagEnd)
    return el

callOptPat = re.compile(r'call\s*opt',re.IGNORECASE)
putOptPat = re.compile(r'put\s*opt',re.IGNORECASE)

def getRowInfo(row) :
    """
    Returns information for a row in a 13F table in the form:
        (cusip, name, value, title, count, putCall)
    where the field values are as given in the table,
    except putCall is 'CALL', 'PUT', or ''.
    """
    cusip = findChildEndingWith(row,'cusip').text.upper().strip()
    name = findChildEndingWith(row,'issuer').text.strip()
    value = findChildEndingWith(row,'value').text.strip()
    title = findChildEndingWith(row,'titleOfClass').text.upper().strip()
    shrsOrPrnEl = findChildEndingWith(row,'shrsOrPrnAmt')
    count = findChildEndingWith(shrsOrPrnEl,'sshPrnamt').text.strip()
    #countType = findChildEndingWith(shrsOrPrnEl,'sshPrnamtType').text.upper()
    putCallEl = findChildEndingWith(row,'putCall')
    if putCallEl is None :
        putCallEl = findChildEndingWith(shrsOrPrnEl,'putCall')
        #if putCallEl is not None :
        #    print('found putcall in shrsOrPrn')
    if putCallEl is not None :
        putCall = putCallEl.text.upper().strip()
    elif callOptPat.search(name) or title.startswith('CALL') or title=='CAL' :
        putCall = 'CALL'
    elif putOptPat.search(name) or title.startswith('PUT') :
        putCall = 'PUT'
    else :
        putCall = ''
    return (cusip, name, value, title, count, putCall)

def parse13FHoldings(accNo, formType=None) :
    """
    Parses a 13F filing, returning the result in the form:
    {
        'period': 'YYYY-MM-DD',
        'acceptDate': 'YYYY-MM-DD',
        'acceptTime': 'HH:MM:SS',
        'cik' : 'DDDDDDDDDD',
        'holdings': [(cusip, name, value, title, count, putCall), ... ]
    }
    where the field values are as given in the table,
    except putCall is 'CALL', 'PUT', or ''.
    """
    info = basicInfo.getSecFormInfo(accNo, formType)
    xmlUrls = [l[-1] for l in info['links'] if l[0].lower().endswith('xml')]
    if len(xmlUrls) == 1 :
        xmlSummTab = utils.downloadSecUrl(xmlUrls[0],toFormat='xml')
        tot = int(findChildSeries(xmlSummTab,['formdata','summarypage','tableentrytotal']).text.strip())
        if tot == 0 :
            print('*** zero total, table not present')
        else :
            print('*** nonzero total, but table not present')
        holdings = []
    else :
        xmlTab = utils.downloadSecUrl(xmlUrls[-1],toFormat='xml')
        tabRows = [tabRow for tabRow in xmlTab
                   if tabRow.tag.lower().endswith('infotable')]
        if len(xmlTab) != len(tabRows) :
            print('*** #rows mismatch',len(xmlTab),'all children',len(tabRows),'table rows')
        if len(tabRows) == 0 :
            print('*** no holdings in table')
        holdings = [getRowInfo(tabRow) for tabRow in tabRows]
    if len(info['ciks']) != 1 :
        print('*** unexpected number of CIKs!=1',info['ciks'])
    return {'period': info['period'],
            'acceptDate': info['acceptDate'],
            'acceptTime': info['acceptTime'],
            'cik': info['ciks'][0],
            'holdings': holdings}

class scraper13F(infoScraper.scraperBase) :
    @utils.delegates(infoScraper.scraperBase.__init__)
    def __init__(self, infoDir=default13FDir, **kwargs) :
        super().__init__(infoDir, '13F-HR', **kwargs)
    def scrapeInfo(self, accNo, formType=None) :
        return parse13FHoldings(accNo, formType), None

Test 13F-HR scraper class:

In [ ]:
dl = dailyList.dailyList(startD='empty')
dl.updateForDays('20210614','20210615')
assert len(dl.getFilingsList(None,'13F-HR')[0])==4, "testing 13F scraper class (daily list count)"

s = scraper13F(startD='empty')
s.updateForDays(dl,'20210614','20210615')
hList = s.infoMap['20210614']['0001104659-21-080656']['holdings']
assert (len(hList)==39
        and hList[0]==('00289Y107', 'Abeona Therapeutics Inc', '5376', 'COM', '2859767', '')
        and hList[-1]==('82835P103', 'SILVERCORP METALS INC', '179', 'COM', '36500', '')
       ), "testing 13F scraper class (parsing holdings table)"

20210614 ### list index 64 filings for 20210614: 4379 * ==========NEW 20210614========== '0001104659-21-080656' '0001104659-21-080840' '0000921895-21-001608' '0000921895-21-001612' 

Generate a combined holdings matrix based on all 13F-HR filings in a quarter:

In [ ]:
#export

def condense13FHoldings(holdings) :
    """
    Converts a list of of stock and option holdings as parsed from the 13F:
        [(cusip, name, value, title, count, putCall), ... ]
    that may have multiple entries per stock into a condensed list that omits
    call/put options and only has one combined entry per stock:
        [(cusip, val, frac) ... ]
    sorted in descending order by value.
    """
    # eliminate options and sort to group holdings by CUSIP (stock identifier):
    holdings = sorted((cusip, float(value))
                      for cusip, name, value, shType, nShares, putCall in holdings
                      if putCall=='')
    # combine into a single entry for each stock with the total for that stock:
    holdings = [(cusip, sum(val for _,val in it))
                for cusip,it in itertools.groupby(holdings, key=lambda x : x[0])]
    # sort to put largest holdings first:
    holdings.sort(key = lambda x : x[1], reverse=True)
    # calculate the fraction of total holdings for each stock:
    totAum = sum(val for _,val in holdings)
    return [(cusip, val, val/totAum if totAum>0.0 else 0.0) for cusip,val in holdings]

def filter13FHoldings(holdings, minAUM=None, minTopNFrac=None, minTopN=10, max13FCount=None,
                      minFrac=0.0, maxFrac=1.0) :
    """
    Filters a condensed list of 13F stock holdings as returned from condense13FHoldings,
    i.e. a list of tuples:
        [(cusip, val, frac) ... ]
    sorted in descending order by value.

    If minTopNFrac is specified, returns None for lists with too small a total fraction
    in the top minTopN stocks.
    If minAUM is specified, returns None for lists with too small a total value.
    If max13FCount is specified, returns None for lists with too many stocks.

    Restricts the returned list to stocks with fraction of total value in [minFrac..maxFrac]
    """
    # return None if the investor is eliminated by any of the options:
    totAum = sum(val for _,val,_ in holdings)
    if ((minAUM is not None
             and minAUM > totAum*1000.0)
        or (minTopNFrac is not None
                and minTopNFrac > sum(frac for _,_,frac in holdings[:minTopN]))
        or (max13FCount is not None
                and len(holdings) > max13FCount)) :
        return None
    # get the final output list, filtered by min/maxFrac
    res = [tup for tup in holdings if minFrac<=tup[2]<=maxFrac]
    return res if len(res)>0 else None

@utils.delegates(filter13FHoldings)
def get13FHoldingsReportList(holdings, cusipNames={}, **kwargs) :
    """
    Converts a list of of stock and option holdings as parsed from the 13F:
        [(cusip, name, value, title, count, putCall), ... ]
    into a semi-formatted list for compact reporting:
        [(cusip, name, val, pctstr) ... ]
    where pctstr is a string in the format N.NN%

    Uses filter13FHoldings to filter the returned list.
    """
    filteredH = filter13FHoldings(condense13FHoldings(holdings), **kwargs)
    if filteredH is None :
        return []
    cusipToNameInForm = dict((cusip,name)
                             for cusip, name, value, shType, nShares, putCall in holdings)
    return [(cusip, cusipNames.get(cusip,cusipToNameInForm[cusip]), val, f'{frac:.2%}')
            for (cusip, val, frac) in filteredH]

def get13FAmendmentType(accNo, formType=None) :
    """
    Gets the amendment type for a 13F-HR/A filing - may be RESTATEMENT or NEW HOLDINGS.
    This turned out to be unreliable (often missing or wrong), so I don't use it to get
    the combined holdings for an investor. Instead I just look at the number of holdings
    in an amendment compared to the previous filing, and treat it as a restatement
    if the new number of holdings is more than half the old number.
    """
    info = basicInfo.getSecFormInfo(accNo, formType)
    xmlUrls = [l[-1] for l in info['links'] if l[0].lower().endswith('xml')]
    xmlSummTab = utils.downloadSecUrl(xmlUrls[0],toFormat='xml')
    coverPage = findChildSeries(xmlSummTab,['formdata','coverpage'])
    isAmendment = findChildEndingWith(coverPage,'isamendment')
    if isAmendment is None or isAmendment.text.strip().lower() not in ['true','yes'] :
        return None
    return findChildSeries(coverPage,['amendmentinfo','amendmenttype']).text.strip()

def indexMap(lis) :
    "Converts a list to a dict mapping item -> index in the list."
    return dict((el,i) for i,el in enumerate(lis))

@utils.delegates(filter13FHoldings)
def getHoldingsMapFrom13Fs(scraped13F, period, allCusipCounter=None, all13FHoldingsMap=None, **kwargs) :
    """
    Consolidate holdings for each CIK based on all 13F filings for a given period into
    a combined map of investor holdings.

    Returns a dict: cik -> {cusip -> pct}

    Restricts to stocks only (no call/put options).

    If allCusipCounter is supplied, updates it to count all investors per stock.
    If all13FHoldingsMaps is supplied, saves a list of all stock holdings for each CIK
    as returned by condense13FHoldings, i.e. a list of tuples:
        all13FHoldingsMap[cik] = [(cusip, val, frac) ... ]
    sorted in descending order by value.

    Uses filter13FHoldings to filter the returned holdings.
    """
    # Map cik to a list [(dateStr, accNo, holdingsList) ... ]
    # of all 13F filings from that cik with the given period.
    cikTo13Fs = collections.defaultdict(list)
    count = 0
    for dStr, accNoToInfo in scraped13F.infoMap.items() :
        for accNo, info in accNoToInfo.items() :
            if info == 'ERROR' :
                print('ERR',accNo)
            elif info['period'] == period :
                cikTo13Fs[info['cik'].lstrip('0')].append((dStr, accNo, info['holdings']))
                count += 1
    print('period',period,'- total of',len(cikTo13Fs),'ciks,',count,'13F filings')
    #
    # Get a consolidated list of positions for each cik. For ciks with multiple filings
    # this may involve combining amended filings. Since the filed amendment type is
    # unreliable I use a simple rule of thumb - if the amendment has more than half
    # as many positions as the previous filing I assume it's a restatement, otherwise
    # I add its positions to the previous filing.
    res = {}
    elimCount = 0
    for cik, cik13FList in cikTo13Fs.items() :
        cik13FList.sort()  # sort by day and then by accession number
        i = 0
        j = 1
        while j < len(cik13FList) :
            if len(cik13FList[j][2]) > len(cik13FList[i][2])//2 :
                # relatively many new positions - assume filing j is a restatement
                i = j
            j += 1
        if j != 1 :
            print('CIK',cik,i,'-',j,[(dStr,accNo,len(holdings))
                                     for dStr,accNo,holdings in cik13FList])
        combHoldings = cik13FList[i][2]
        while i+1 < j :
                i += 1
                combHoldings = combHoldings + cik13FList[i][2]
        fullPosList = condense13FHoldings(combHoldings)
        if all13FHoldingsMap is not None :
            all13FHoldingsMap[cik] = fullPosList
        if allCusipCounter is not None :
            allCusipCounter.update(cusip for cusip,_,_ in fullPosList)
        posList = filter13FHoldings(fullPosList, **kwargs)
        if posList is not None :
            res[cik] = dict((cusip,frac) for cusip,_,frac in posList)
        else :
            # print('eliminated',fullPosList)
            elimCount += 1
    print('calculated holdings map from 13F forms for',period)
    print('13F investor filters:',kwargs)
    print(elimCount,'investors eliminated by filters')
    return res

def addHoldingsMap(holdingsMap, extraHoldingsMap) :
    """
    Adds positions in extraHoldingsMap to holdingsMap.
    Each argument is a dict: cik -> {cusip -> pct}
    but extraHoldingsMap may contain ciks and cusips not in holdingsMap.
    """
    for cik,extraPosMap in extraHoldingsMap.items() :
        if cik not in holdingsMap :
            holdingsMap[cik] = {}
        posMap = holdingsMap[cik]
        for cusip,frac in extraPosMap.items() :
            posMap[cusip] = posMap.get(cusip,0.0) + frac

def printRemoveStocksMessage(cusipsToRemove, delCount, msg) :
    print(msg,'- removed',len(cusipsToRemove)-delCount,'stocks')
    return len(cusipsToRemove)

def holdingsMapToMatrix(holdingsMap, allCusipCounter,
                        minAllInvsPerStock=None, maxAllInvsPerStock=None,
                        minMatStocksPerInv=None, maxMatStocksPerInv=None,
                        minMatInvsPerStock=None, maxMatInvsPerStock=None,
                        cusipFilter=None, dtype=np.float64) :
    """
    Converts a holdings map: cik -> {cusip -> frac} into a matrix.

    Returns mat, ciks, cusips where mat is a matrix of shape (len(ciks), len(cusips))
    in which each row has the fractions held by the corresponding cik in each cusip.

    If minAllInvsPerStock or maxAllInvsPerStock is specified, then allCusipCounter should be a
    Counter giving a count of all investors that have any position in each stock, and the result
    will be filtered based on this count. Otherwise allCusipCounter can be None.

    If minMatStocksPerInv is specified, restricts to investors with at least that many stocks
    in the returned matrix; likewise, maxMatStocksPerInv can be used to give an upper bound.

    If minMatInvsPerStock is specified, restricts to stocks with at least that many investors
    in the returned matrix; likewise, maxMatInvsPerStock can be used to give an upper bound.

    If cusipFilter is specified, this should be a function that returns True for cusips to keep.

    dtype specifies the type of the returned matrix.
    """
    invCount = len(holdingsMap)
    print('starting investor count:',invCount)
    if minMatStocksPerInv is None and maxMatStocksPerInv is None :
        print('not limiting number of stocks per investor in the ratings matrix')
    else :
        if minMatStocksPerInv is not None :
            print('requiring at least',minMatStocksPerInv,'stocks per investor in the ratings matrix')
            holdingsMap = dict((cik,posMap)
                               for cik,posMap in holdingsMap.items()
                               if len(posMap) >= minMatStocksPerInv)
            print('- removed',invCount-len(holdingsMap),'investors,',len(holdingsMap),'remaining')
            invCount = len(holdingsMap)
        if maxMatStocksPerInv is not None :
            print('requiring at most',maxMatStocksPerInv,'stocks per investor in the ratings matrix')
            holdingsMap = dict((cik,posMap)
                               for cik,posMap in holdingsMap.items()
                               if len(posMap) <= maxMatStocksPerInv)
            print('- removed',invCount-len(holdingsMap),'investors,',len(holdingsMap),'remaining')
            invCount = len(holdingsMap)
    matCusipCounter = collections.Counter()
    for posMap in holdingsMap.values() :
        matCusipCounter.update(posMap.keys())
    print('starting stock count:',len(matCusipCounter))
    cusipsToRemove = set()
    delCount = 0
    if (minAllInvsPerStock is None and maxAllInvsPerStock is None 
            and minMatInvsPerStock is None and maxMatStocksPerInv is None) :
        print('not limiting number of investors per stock')
    else :
        if minAllInvsPerStock is not None :
            cusipsToRemove.update(cusip for cusip in matCusipCounter
                                  if allCusipCounter[cusip] < minAllInvsPerStock)
            delCount = printRemoveStocksMessage(cusipsToRemove,delCount,
                        f'requiring at least {minAllInvsPerStock} ALL investors per stock')
        if maxAllInvsPerStock is not None :
            cusipsToRemove.update(cusip for cusip in matCusipCounter
                                  if allCusipCounter[cusip] > maxAllInvsPerStock)
            delCount = printRemoveStocksMessage(cusipsToRemove,delCount,
                        f'requiring at most {maxAllInvsPerStock} ALL investors per stock')
        if minMatInvsPerStock is not None :
            cusipsToRemove.update(cusip for cusip,count in matCusipCounter.items()
                                  if count < minMatInvsPerStock)
            delCount = printRemoveStocksMessage(cusipsToRemove,delCount,
                        f'requiring at least {minMatInvsPerStock} investor ratings per stock')
        if maxMatStocksPerInv is not None :
            cusipsToRemove.update(cusip for cusip,count in matCusipCounter.items()
                                  if count > maxMatStocksPerInv)
            delCount = printRemoveStocksMessage(cusipsToRemove,delCount,
                        f'requiring at most {maxMatStocksPerInv} investor ratings per stock')
    if cusipFilter is not None :
        cusipsToRemove.update(cusip for cusip in matCusipCounter
                              if not cusipFilter(cusip))
        delCount = printRemoveStocksMessage(cusipsToRemove,delCount,'applying CUSIP filter')
    cusips = sorted(set(matCusipCounter.keys()) - cusipsToRemove)
    if delCount > 0 :
        print('removed a total of',delCount,'stocks,',len(cusips),'remaining')
    ciks = sorted(cik.zfill(10) for cik,posMap in holdingsMap.items()
                  if 1 <= len(set(posMap.keys())-cusipsToRemove))
    print('removed',invCount-len(ciks),'investors with no remaining positions')
    print(f'final counts: {len(ciks):,} investors; {len(cusips):,} stocks;',end=' ')
    cikToRow = indexMap(ciks)
    cusipToCol = indexMap(cusips)
    mat = np.zeros((len(ciks), len(cusips)), dtype=dtype)
    count = 0
    for cik,posMap in holdingsMap.items() :
        cikRow = cikToRow.get(cik.zfill(10))
        if cikRow is None :
            continue
        for cusip,frac in posMap.items() :
            cusipCol = cusipToCol.get(cusip)
            if cusipCol is not None :
                mat[cikRow, cusipCol] = frac
                count += 1
    print(f'{count:,} positions')
    return mat, ciks, cusips    

qStartEnds = ['0101','0401','0701','1001','0101']
qPeriods = ['-03-31','-06-30','-09-30','-12-31']
def getPeriodAndNextQStartEnd(y, qNo) :
    """
    Returns the 13F period date for a given year and quarter number (this is the
    last day in the quarter), along with the start and end dateStrs for the next
    quarter (this is the date range when the 13Fs for this year should be filed).
    Quarters are numbered 1-4.
    """
    nextY = y+1 if qNo==4 else y
    nextQNo = 1 if qNo==4 else qNo+1
    return (str(y)+qPeriods[qNo-1],
            {'startD' : str(nextY) + qStartEnds[nextQNo-1],
             'endD' : str(nextY+1 if nextQNo==4 else nextY) + qStartEnds[nextQNo]})

@utils.delegates(filter13FHoldings, holdingsMapToMatrix)
def getNSSForQ(y, qNo, allCusipCounter=None,
               include13F=True, all13FHoldingsMap=None, extraHoldingsMaps=[],
               **kwargs) :
    """
    Calculates a matrix of investor holdings for a quarter, based on all 13F filings filed
    during the succeeding quarter.

    Returns mat, ciks, cusips where mat is a matrix of shape (len(ciks), len(cusips))
    in which each row has the fractions held by the corresponding cik in each cusip.

    If supplied, all13FHoldingsMap should be a dict, and it will be updated with a full sorted
    holdings list for each CIK:
        all13FHoldingsMap[cik] = [(cusip, val, frac) ... ]
    without regard to the min/max options supplied to restrict the returned holdings map.

    Optionally adds holdings from a list of extraHoldingsMaps (used for 13G/13D filings).

    Uses filter13FHoldings and holdingsMapToMatrix to filter the returned matrix.
    """
    if include13F :
        period, nextQStartEnd = getPeriodAndNextQStartEnd(y,qNo)
        holdingsMap = utils.callDelegated(getHoldingsMapFrom13Fs, kwargs,
                                          scraper13F(**nextQStartEnd), period,
                                          allCusipCounter=allCusipCounter,
                                          all13FHoldingsMap=all13FHoldingsMap)
    else :
        holdingsMap = {}
    for extraHoldingsMap in extraHoldingsMaps :
        addHoldingsMap(holdingsMap,extraHoldingsMap)
    return utils.callDelegated(holdingsMapToMatrix, kwargs,
                               holdingsMap, allCusipCounter)

convScreenSettings = dict(  # conviction stock screen settings
    minFrac=0.13, maxFrac=0.6, minAUM=7.5e7,
    minMatStocksPerInv=3, minMatInvsPerStock=2,
)
def saveConvMatrixPy2(y, qNo) :
    """
    Save a matrix of 13F conviction positions only for the given quarter,
    in a format readable by the BW Python2 code.
    """
    mat, ciks, cusips = getNSSForQ(y, qNo, allCusipCounter=collections.Counter(),
                                   **convScreenSettings)
    ciks = [cik.encode(encoding='ascii',errors='ignore') for cik in ciks]
    cusips = [cusip.encode(encoding='ascii',errors='ignore') for cusip in cusips]
    m = ([[('0' if el==0.0 else str(el)).encode(encoding='ascii') for el in row] for row in mat],
         ciks, indexMap(ciks), cusips, indexMap(cusips))
    fPath = os.path.join(utils.stockDataRoot,f'Conv{y}Q{qNo}.pkl')
    print('saving to',fPath)
    utils.pickSave(fPath, m, fix_imports=True, protocol=2)

Test generating a combined holdings matrix:

In [ ]:
assert getPeriodAndNextQStartEnd(2020,1)==('2020-03-31', {'startD': '20200401', 'endD': '20200701'}),"13F qstart/end"
assert getPeriodAndNextQStartEnd(2020,4)==('2020-12-31', {'startD': '20210101', 'endD': '20210401'}),"13F qstart/end"

m, ciks, cusips = holdingsMapToMatrix(getHoldingsMapFrom13Fs(s,'2021-03-31'),None)
assert (ciks==['0001325083', '0001867040']
        and cusips[:2]==['00289Y107','003279730'] and cusips[-2:]==['G11196105','M40527109']
        and m.shape==(2,42) and abs(m[0,0]-0.03502463)<1e-6 and abs(m[1,13]-0.9675410812420477)<1e-6
       ), "combined holdings matrix"

min stock fraction of portfolio 0.0
max stock fraction of portfolio 1.0
period 2021-03-31 - total of 2 ciks, 2 13F filings
not limiting number of investors per stock
final counts: 2 investors; 42 stocks; 42 positions


In [ ]:
# mat = saveConvMatrixPy2(2021, 2, minAUM=1e8)

min stock fraction of portfolio 0.13
max stock fraction of portfolio 0.4
min stocks per investor 3
max stocks per investor 500
min AUM (total portfolio value) 100000000.0
period 2021-06-30 - total of 5922 ciks, 6057 13F filings
CIK 1755670 1 - 2 [('20210708', '0001755670-21-000004', 396), ('20210804', '0001755670-21-000005', 509)]
CIK 1842560 1 - 2 [('20210709', '0001842560-21-000003', 332), ('20210709', '0001842560-21-000004', 332)]
CIK 1353395 1 - 2 [('20210712', '0001085146-21-001901', 260), ('20210713', '0001085146-21-001945', 154)]
CIK 1092903 1 - 2 [('20210713', '0001096906-21-001592', 708), ('20210805', '0001096906-21-001852', 708)]
CIK 1840740 1 - 2 [('20210720', '0001840740-21-000007', 52), ('20210720', '0001840740-21-000008', 52)]
CIK 1609674 1 - 2 [('20210721', '0001609674-21-000004', 175), ('20210816', '0001609674-21-000005', 176)]
CIK 1666786 1 - 2 [('20210721', '0001606587-21-000807', 188), ('20210721', '0001606587-21-000811', 188)]
CIK 1767080 0 - 2 [('20210721', '000176

CIK 1844873 2 - 3 [('20210813', '0001844873-21-000004', 81), ('20210816', '0001844873-21-000006', 81), ('20210817', '0001844873-21-000007', 81)]
CIK 1869028 1 - 2 [('20210813', '0001398344-21-016180', 2), ('20210823', '0001398344-21-016691', 2)]
CIK 898427 1 - 2 [('20210813', '0000898427-21-000016', 2347), ('20210813', '0000898427-21-000017', 2351)]
CIK 1084580 1 - 2 [('20210816', '0001085146-21-002551', 1648), ('20210820', '0001085146-21-002619', 1626)]
CIK 1144208 1 - 2 [('20210816', '0001085146-21-002607', 686), ('20210818', '0001085146-21-002615', 685)]
CIK 1212897 0 - 2 [('20210816', '0001315863-21-000701', 18), ('20210817', '0001315863-21-000708', 1)]
CIK 1224962 1 - 2 [('20210816', '0001012975-21-000320', 149), ('20210817', '0001012975-21-000326', 150)]
CIK 1301050 1 - 2 [('20210816', '0001567619-21-015630', 49), ('20210816', '0001567619-21-015814', 57)]
CIK 1393818 1 - 2 [('20210816', '0000950123-21-011566', 623), ('20210818', '0000950123-21-011650', 624)]
CIK 1418814 0 - 2 [('

In [ ]:
# cusipNames = utils.pickLoad(os.path.join(utils.stockDataRoot,'cusipMap.pkl'))
# print(len(cusips),'CUSIPs')
# cn = [cusipNames.get(cusip,'CUSIP-'+cusip) for cusip in cusips]
# fundPats = [re.compile(patStr,re.IGNORECASE)
#            for patStr in [r'\b'+fundStr+r'\b' for fundStr in ['etf','ishares','spdr']]
#                         + [r'^'+fundStr for fundStr in ['janus','pimco','schwab','vanguard','berkshire']]]
# len([c for c in cn if any(fundPat.search(c) for fundPat in fundPats)])

117

In [ ]:
#hide
# uncomment and run to regenerate all library Python files
# from nbdev.export import notebook2script; notebook2script()